# Generate Unseen Test Dataset

# Add Action

# Load OpenAI API key

In [1]:
from datetime import datetime as dt
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
import json
import re

import random
import time
import re
from tqdm import tqdm

In [2]:
from pathlib import Path
from dotenv import load_dotenv
import os

cwd = Path.cwd()
env_path = cwd.parent.parent.parent
print(env_path)

load_dotenv(dotenv_path=env_path / '.env')
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key: Found")
else:
    print("API Key not found. Please ensure it is in the .env file.\n\n")

client = OpenAI()

C:\Users\richa\OneDrive\RChai\Documents\NUS_ISS_Prjs\4_PLP
OpenAI API Key: Found


In [3]:
# List of banking products
BANKING_PRODUCTS_LIST = ["savings account", "current account", "account", "loan", "fixed deposit", "forex account", "mobile app", "website", "atm", "credit card", "debit card", "card", "others"] 

ACTIONS_LIST = ['apply', 'access', 'activate', 'block', 'cancel', 'close', 'deposit', 'dispute', 'earn', 'exchange', 'find', 'inquire', 'link' ,'open', 'pay', 'receive', 'redeem', 'refund', 'renew', 'report', 'reset', 'retrieve', 'schedule', 'select', 'transfer', 'unblock', 'update', 'unlink', 'verify', 'withdraw', 'unknown']

# List of sentiment types
SENTIMENTS_LIST = ["very negative", "negative", "neutral", "positive", "very positive"]

In [4]:
# Helper function to generate a dialog prompt
def generate_dialog_prompt(product, action, sentiment, n_turns):
    return f"Generate a conversation between a customer service executive for a bank and a customer regarding a {product} where the customer sentiment is {sentiment}. The conversation should have around {n_turns} turns and should reflect the customer intent of {action} {product}."

In [5]:
# Generate multi-turn dialogs
def generate_multi_turn_dialog(product, action, sentiment, n_turns=5):
    prompt = generate_dialog_prompt(product, action, sentiment, n_turns)
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",  
            messages=[
                {"role": "system", "content": "You are a helpful and professional customer service agent for a bank."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,  # adjust max_tokens as needed
            n=1
        )

        conversation = response.choices[0].message.content        
        return conversation
    except Exception as e:
        print(f"Error generating dialog: {e}")
        return None

In [6]:
# Main function to generate N unique dialogs
def generate_bank_conversations(num_dialogs=1, n_turns=3):
    dialogs = []
    products = []
    actions = []
    sentiments = []

    for i in tqdm(range(num_dialogs)):    
        # Randomly choose a product and sentiment
        product = random.choice(BANKING_PRODUCTS_LIST)
        action = random.choice(ACTIONS_LIST)
        sentiment = random.choice(SENTIMENTS_LIST)
        
        # Generate dialog
        dialog = generate_multi_turn_dialog(product, action, sentiment, n_turns)
        if dialog:
            dialogs.append(dialog)
            products.append(product)
            actions.append(action)
            sentiments.append(sentiment)
        
        # Add a short delay to avoid rate-limiting issues with API calls
        time.sleep(1.5)  # Adjust if needed
        
    return dialogs, products, actions, sentiments

In [7]:
def split_dialog(dlg_id, in_text, pdt, action, sentiment):
    # Split the dialog into segments
    dialog_segments = re.split(r'\*\*(.*?)\*\*', in_text)

    dialog_id = []
    speaker = []
    text = []
    dlg_product = []
    dlg_action = []
    dlg_sentiment = []
    # Loop through the segments to create pairs
    for i in range(1, len(dialog_segments), 2):
        dialog_id.append(dlg_id)
        dlg_product.append(pdt)
        dlg_action.append(action)
        dlg_sentiment.append(sentiment)
        
        speaker.append(dialog_segments[i].strip(': '))
        text.append(dialog_segments[i + 1].strip().replace('\n', ' '))
        
    turn = list(range(len(speaker)))
    return dialog_id, turn, speaker, text, dlg_product, dlg_action, dlg_sentiment

In [8]:
# Save the generated dialogs to a text file

def save_dialogs_to_file(dialogs, products, actions, sentiments, filename="multi_turn_dialogs.txt"):
    # save multi-turn dialog to csv and parquet
    now = dt.now()
    dlg_prefix = str(now.year).zfill(4) + '-' + str(now.month).zfill(2) + '-' + str(now.day).zfill(2) + '__dlg_'

    dlg_ids = []
    turns = []
    speakers = []
    texts = []
    dlg_pdt = []
    dlg_action = []
    dlg_sentiment = []
    
    for i, dialog in enumerate(dialogs):
        dlg_prefix_id = dlg_prefix + str(i)
        # d, ti, s, u, p, a, st i.e. dlg_id, turn, speaker, utterance, product, action, sentiment
        d, t, s, u, p, a, st = split_dialog(dlg_prefix_id, dialog, products[i], actions[i], sentiments[i])
        dlg_ids.extend(d)
        turns.extend(t)
        speakers.extend(s)
        texts.extend(u)
        dlg_pdt.extend(p)
        dlg_action.extend(a)
        dlg_sentiment.extend(st)
        
    data_0 = {
        "dlg_id": dlg_ids,
        "turn": turns,
        "speaker": speakers,
        "text": texts,
        "overall_dlg_pdt": dlg_pdt,
        "overall_dlg_action": dlg_action,
        "overall_dlg_sentiment": dlg_sentiment
    }
    dft = pd.DataFrame(data_0)    
    dft['text'] = dft['text'].astype(str)
    dft.to_csv('Generated_dialogs.csv', index=False, encoding='utf-8')
    dft.to_parquet('Generated_dialogs.parquet', index=False)   
    return dft

In [9]:
# Run the script
num_dialogs = 2000
n_turns = 5
DIALOGS, PRODUCTS, ACTIONS, SENTIMENTS = generate_bank_conversations(num_dialogs, n_turns)

df = save_dialogs_to_file(DIALOGS, PRODUCTS, ACTIONS, SENTIMENTS)

100%|███████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [3:23:00<00:00,  6.09s/it]


In [10]:
df.head(10)

dlg_id  turn                     speaker  \
0  2024-10-14__dlg_0     0  Customer Service Executive   
1  2024-10-14__dlg_0     1                    Customer   
2  2024-10-14__dlg_0     2  Customer Service Executive   
3  2024-10-14__dlg_0     3                    Customer   
4  2024-10-14__dlg_0     4  Customer Service Executive   
5  2024-10-14__dlg_0     5                    Customer   
6  2024-10-14__dlg_1     0                    Customer   
7  2024-10-14__dlg_1     1  Customer Service Executive   
8  2024-10-14__dlg_1     2                    Customer   
9  2024-10-14__dlg_1     3  Customer Service Executive   

                                                text overall_dlg_pdt  \
0  Hello, thank you for calling [Bank Name]. My n...             atm   
1  Hi Alex! I'm doing great, thanks for asking. I...             atm   
2  I'm glad to hear you're excited about your new...             atm   
3  Sure! I just moved to Maple Lakes. It’s such a...             atm   
4  Maple Lakes sounds wonderful! Let me just chec...             atm   
5  That's perfect, thanks so much, Alex! I'm real...             atm   
6  Hi, I've just about had it with this bank. I'm...          others   
7  I'm sorry to hear that you're feeling this way...          others   
8  Well, for starters, the recent service fees ha...          others   
9  I understand how concerning unexpected fees ca...          others   

  overall_dlg_action overall_dlg_sentiment  
0               find         very positive  
1               find         very positive  
2               find         very positive  
3               find         very positive  
4               find         very positive  
5               find         very positive  
6           activate         very negative  
7           activate         very negative  
8           activate         very negative  
9           activate         very negative

# Important Notes

# TEST

In [11]:
# for row in df[:10].itertuples():
#     print(row.text)
#     print()